In [7]:
from pyspark.sql import *

In [8]:
from pyspark.sql.functions import *

In [9]:
path = '/data/'

In [10]:
geolocation = spark.read.csv(path+'olist_geolocation_dataset.csv',header=True,inferSchema = True)
order_items = spark.read.csv(path+'olist_order_items_dataset.csv',header=True,inferSchema = True)
payments = spark.read.csv(path+'olist_order_payments_dataset.csv',header=True,inferSchema = True)
reviews = spark.read.csv(path+'olist_order_reviews_dataset.csv',header=True,inferSchema = True)
orders = spark.read.csv(path+'olist_orders_dataset.csv',header=True,inferSchema = True)
products = spark.read.csv(path+'olist_products_dataset.csv',header=True,inferSchema = True)
seller = spark.read.csv(path+'olist_sellers_dataset.csv',header=True,inferSchema = True)
customer = spark.read.csv(path+'olist_customers_dataset.csv',header=True,inferSchema = True)

In [11]:
customer.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

**Checking for Null Values

In [12]:
def null_values(df,df_name):
    print(f'Looking at {df_name}')
    df.select([count(when(col(c).isNull(), 1)).alias(c)for c in df.columns]).show()

In [13]:
null_values(geolocation,'geolocation')

Looking at geolocation


+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+



In [14]:
null_values(order_items,'order_items')

Looking at order_items


+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
|       0|            0|         0|        0|                  0|    0|            0|
+--------+-------------+----------+---------+-------------------+-----+-------------+



In [15]:
null_values(payments,'payments')

Looking at payments


+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



In [16]:
null_values(reviews,'reviews')

Looking at reviews


+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|        1|    2236|        2380|               92157|                 63079|                8764|                   8785|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+



In [17]:
null_values(orders,'orders')

Looking at orders


+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [18]:
null_values(products,'products')

Looking at products
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|         0|                  610|                610|                       610|               610|               2|                2|                2|               2|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+



In [19]:
null_values(seller,'seller')

Looking at seller
+---------+----------------------+-----------+------------+
|seller_id|seller_zip_code_prefix|seller_city|seller_state|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+



In [20]:
null_values(customer,'customer')

Looking at customer
+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



**Handling Missing Values

In [21]:
cleaned_orders = orders.na.drop(
    subset=['order_id', 'customer_id', 'order_status']
)


In [22]:
cleaned_orders.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

**Filling null values

In [23]:
cleaned_orders = orders.fillna({'order_delivered_customer_date':'9999-12-31'})

In [24]:
cleaned_orders.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

**Impute null values

In [25]:
payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [26]:
payments.filter(col('payment_value').isNull()).count()

0

In [27]:
payments_w_null = payments.withColumn('payment_value', when(col('payment_value')!=99.33,col('payment_value')).otherwise(lit(None)))

In [28]:
payments_w_null.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [32]:
from pyspark.ml.feature import Imputer

In [33]:
imputer = Imputer(inputCols = ['payment_value'], outputCols = ['payment_value_imputed']).setStrategy('median')

In [34]:
cleaned_payments = imputer.fit(payments_w_null).transform(payments_w_null)

In [35]:
cleaned_payments.show() ##showing close to 99.33

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|                100.0|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               

In [36]:
cleaned_payments.count()

103886

**Standardizing the Format

In [37]:
def print_schema(df,df_name):
    print(f'Looking at {df_name}-')
    print(df.printSchema())

In [38]:
print_schema(geolocation,'geolocation')

Looking at geolocation-
root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)

None


In [39]:
print_schema(customer,'customer')

Looking at customer-
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

None


In [40]:
print_schema(order_items,'order_items')

Looking at order_items-
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)

None


In [41]:
print_schema(payments,'payments')

Looking at payments-
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)

None


In [42]:
print_schema(reviews,'reviews')

Looking at reviews-
root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)

None


In [43]:
print_schema(orders,'orders')

Looking at orders-
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

None


In [44]:
print_schema(products,'products')

Looking at products-
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)

None


In [45]:
print_schema(seller,'seller')

Looking at seller-
root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)

None


In [46]:
print_schema(customer,'customer')

Looking at customer-
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

None


In [47]:
cleaned_orders.show(7)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [48]:
cleaned_orders = cleaned_orders.withColumn('order_purchase_timestamp', to_date(col('order_purchase_timestamp'))) # example of changing the 1 column into date

In [49]:
cleaned_orders.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|              2017-10-02|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|              2018-07-24|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [51]:
cleaned_payments = payments.withColumn('payment_type',when(col('payment_type') == 'boleto', 'Bank Transfer').when(col('payment_type') == 'credit_card', 'Credit Card')\
                                      .when(col('payment_type') == 'debit_card', 'Debit Card')\
                                      .when(col('payment_type') == 'voucher', 'Voucher')\
                                      .when(col('payment_type') == 'Null', 'Undefined'))  #Sample of standardizing values in Schema

In [52]:
cleaned_payments.show()

+--------------------+------------------+-------------+--------------------+-------------+
|            order_id|payment_sequential| payment_type|payment_installments|payment_value|
+--------------------+------------------+-------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1|  Credit Card|                   8|        99.33|
|a9810da82917af2d9...|                 1|  Credit Card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1|  Credit Card|                   1|        65.71|
|ba78997921bbcdc13...|                 1|  Credit Card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1|  Credit Card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1|  Credit Card|                   2|        96.12|
|771ee386b001f0620...|                 1|  Credit Card|                   1|        81.16|
|3d7239c394a212faa...|                 1|  Credit Card|                   3|        51.84|

In [53]:
cleaned_payments.groupBy('payment_type').count().show()

+-------------+-----+
| payment_type|count|
+-------------+-----+
|  Credit Card|76795|
|         NULL|    3|
|Bank Transfer|19784|
|   Debit Card| 1529|
|      Voucher| 5775|
+-------------+-----+



**Enforcing Schema Sample

In [54]:
customer.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [55]:
customer.show(10)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [56]:
cleaned_customer = customer.withColumn('customer_zip_code_prefix',col('customer_zip_code_prefix').cast('string'))

In [57]:
cleaned_customer.printSchema() 

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



**Removing Duplicates

In [59]:
cleaned_customer = cleaned_customer.dropDuplicates(['customer_id'])

In [61]:
cleaned_customer.count()

99441

**Order with Details

In [75]:
detailed_orders = cleaned_orders.join(order_items,'order_id','left').join(cleaned_payments,'order_id','left')

In [76]:
detailed_orders.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+------------------+-------------+--------------------+-------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|payment_sequential| payment_type|payment_installments|payment_value|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+----

In [73]:
orders.show(2)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
+--------------------+--------------------+--

**Feature Engineering - Showing total payment value for one order id

In [83]:
orders_value = detailed_orders.groupBy('order_id').agg(sum('payment_value').alias('total')).orderBy('total', ascending=False)

In [88]:
orders_value.show()

+--------------------+------------------+
|            order_id|             total|
+--------------------+------------------+
|03caa2c082116e1d3...|         109312.64|
|ab14fdcfbe524636d...| 45256.00000000001|
|1b15974a0141d54e3...|44048.000000000015|
|2cc9089445046817a...|          36489.24|
|e8fa22c3673b1dd17...|30185.999999999993|
|736e1922ae60d0d6a...|          29099.52|
|9aec4e1ae90b23c7b...|           22346.6|
|71dab1155600756af...|21874.049999999996|
|912343626f370ead5...|          19457.04|
|4412d97cb2093633a...|          19174.38|
|9f5054bd9a3c71702...|18667.000000000004|
|428a2f660dc84138d...|          18384.75|
|3a213fcdfe7d98be7...|          17786.88|
|f80549a97eb203e15...|           17671.0|
|cf4659487be50c0c3...|          17069.76|
|be382a9e1ed251281...|16313.600000000002|
|637617b3ffe9e2f7a...|14963.639999999998|
|c52c7fbe316b5b9d5...|14577.569999999998|
|f60ce04ff8060152c...|14401.000000000002|
|73c8ab38f07dc9438...|14196.280000000004|
+--------------------+------------

***Advance Transformation, checking and removing outliers

In [92]:
quantiles = order_items.approxQuantile('price',[0.01,0.99],0.0)
low_cutoff,high_cutoff = quantiles[0],quantiles[1]

In [93]:
low_cutoff,high_cutoff

(9.99, 890.0)

In [94]:
order_items.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [104]:
cleaned_order_items = order_items.filter((col('price') >= low_cutoff) & (col('price') <= high_cutoff))

In [106]:
cleaned_order_items.show(10)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [ ]:
**Categorizing Values

In [138]:

cleaned_products = products.withColumn(
    'product_s_cat',
    when(col('product_weight_g') < 250, 'small')
    .when(col('product_weight_g') < 600, 'medium')
    .otherwise('large')
)


In [139]:
cleaned_products.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+-------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_s_cat|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+-------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|           small|               16|               10|              14|        large|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|           large|               30|               18|              20|        large|
|96bd76ec8810374ed...|   

**Total Revenue per Seller

In [143]:
detailed_orders.groupBy('seller_id').agg(sum('payment_value').alias('total')).orderBy('total',ascending=False).show()

+--------------------+------------------+
|           seller_id|             total|
+--------------------+------------------+
|7c67e1448b00f6e96...|         507166.91|
|1025f0e2d44d7041d...| 308222.0399999999|
|4a3ca9315b744ce9f...|301245.26999999984|
|1f50f920176fa81da...| 290253.4199999998|
|53243585a1d6dc264...|284903.07999999996|
|da8622b14eb17ae28...| 272219.3200000001|
|4869f7a5dfa277a7d...| 264166.1199999999|
|955fee9216a65b617...| 236322.3000000002|
|fa1c13f2614d7b5c4...|         206513.23|
|7e93a43ef30c4f03f...|185134.21000000005|
|6560211a19b47992c...|179657.74999999974|
|7a67c85e85bb2ce85...|169030.80000000008|
|                NULL|162591.95000000004|
|25c5c91f63607446a...|         160534.74|
|a1043bafd471dff53...|154356.91000000003|
|46dc3b2cc0980fb8e...|148864.34000000003|
|b37c4c02bda3161a7...|         145319.04|
|620c87c171fb2a6dd...|145267.94999999987|
|cc419e0650a3c5ba7...|141309.57999999978|
|5dceca129747e92ff...|132974.41999999998|
+--------------------+------------

In [151]:
!hadoop fs -ls /data/

Found 10 items
drwxr-xr-x   - root      hadoop          0 2025-12-24 12:44 /data/dataproc
-rw-r--r--   2 iantrisdc hadoop    9033957 2025-12-23 06:13 /data/olist_customers_dataset.csv
-rw-r--r--   2 iantrisdc hadoop   61273883 2025-12-23 06:13 /data/olist_geolocation_dataset.csv
-rw-r--r--   2 iantrisdc hadoop   15438671 2025-12-23 06:13 /data/olist_order_items_dataset.csv
-rw-r--r--   2 iantrisdc hadoop    5777138 2025-12-23 06:13 /data/olist_order_payments_dataset.csv
-rw-r--r--   2 iantrisdc hadoop   14451670 2025-12-23 06:13 /data/olist_order_reviews_dataset.csv
-rw-r--r--   2 iantrisdc hadoop   17654914 2025-12-23 06:13 /data/olist_orders_dataset.csv
-rw-r--r--   2 iantrisdc hadoop    2379446 2025-12-23 06:13 /data/olist_products_dataset.csv
-rw-r--r--   2 iantrisdc hadoop     174703 2025-12-23 06:13 /data/olist_sellers_dataset.csv
-rw-r--r--   2 iantrisdc hadoop       2613 2025-12-23 06:13 /data/product_category_name_translation.csv


In [153]:
detailed_orders.write.mode('overwrite').parquet('/data/dataproc')